### TP3


1. Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $(n,m)\,$na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $n$ e $m$ à sua escolha.
    Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos em `BitVec`  (apresentado no TP2).

In [1]:
from z3 import *
from pysmt.shortcuts import *
import itertools 

<img src="automato.png" alt="autómato" style="width: 600px;"/>

#### Função declare(t, N):

   A seguinte função cria a $t$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [9]:
'''
def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)
    
    return state
'''

def genState(vars,s,i,n):
    state = {}
    state['pc'] = Int('pc'+'!'+s+str(i))
    for v in vars[1:]:
        state[v] = BitVec(v+'!'+s+str(i),n)
    return state

#### Função init(state, prob, N):

    state - Dicionário de variáveis de estado
    a - valor associado ao 1º número a multiplicar
    b - valor associado ao 2º número a multiplicar

   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [10]:
def init(state, a, b):
    return And(state['pc']== 0, state['x'] == a, state['y'] == b, state['z'] == 0)

#### Função trans(curr, prox, N):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    n - Número de bits 
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [11]:
def error(state):
    return Or(state['pc'] == 4, state['pc'] == 6)

def trans(curr, prox, n):
    same_values = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'],
        prox['z'] == curr['z']    
    )
    
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        same_values
    )
    
    # y = 0
    t1 = And(
        curr['y'] == 0,
        curr['pc'] == 1,
        prox['pc'] == 5,
        same_values
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 1,
        curr['pc'] == 1,
        prox['pc'] == 2,
        same_values
    )
    
    
    # y != 0 ^ even(y)
    t3 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 0,
        curr['pc'] == 1,
        prox['pc'] == 3,
        same_values
    )

    

    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia_left = And(
        prox['x'] == curr['x'] << BitVecVal(1, n),
        prox['y'] == curr['y'] >> BitVecVal(1, n),
        prox['z'] == curr['z'],
        
        curr['pc'] == 3,
        
        Or(
            And(ULT(curr['x'], prox['x']), prox['pc'] == 1),  # curr['x'] < prox['x']  - não há overflow
            And(UGE(curr['x'], prox['x'] ), prox['pc'] == 4),  # curr['x'] >= prox['x'] - há overflow
        )
    )
    
    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia_right = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'] - BitVecVal(1,n),
        prox['z'] == curr['z'] + curr['x'],
        
        curr['pc'] == 2,
        
        Or(
            And(UGT(prox['z'], curr['z']), prox['pc'] == 1),  # curr['x'] < prox['x']  - não há overflow
            And(ULT(prox['z'], curr['z']), prox['pc'] == 6),  # curr['x'] >= prox['x'] - há overflow
        )
    )
    
    
    # caso de paragem no overflow e no estado final
    stop_case = And(
        prox['pc'] == curr['pc'],
        same_values,
        
        Or(
            And(curr['pc'] == 4, prox['pc'] == 4),
            And(curr['pc'] == 5, prox['pc'] == 5),
            And(curr['pc'] == 6, prox['pc'] == 6)
        )
    )
    
    
    return Or(t0, t1, t2, t3, stop_case, magia_left, magia_right)

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço
    n - Número de bits
    a - Valor a ser multiplicado
    b - Valor a ser multiplicado

A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado, de um predicado que testa se um estado é inicial, um número positivo para gerar um possível traço de execução do programa de tamanho `k` , com `N` bits, multiplicando `a` por `b`.

In [12]:
def gera_traco(vars,init,trans, error, k, n, a, b):

    s = Solver()
    
    
    X = [genState(vars, 'X', i, n) for i in range(k+1)]
    I = init(X[0], a, b)
    Tks = [trans(X[i], X[i+1], n) for i in range(k)]

    s.add([I, And(Tks)])
     
    check = s.check()
    if check == sat:
        m = s.model()
        #print(m)
        #print(X)
        for i in range(k):
            print("Passo ", i)
            for v in X[i]:
                
                print(v, "=", m[X[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(['pc', 'x', 'y', 'z'],init,trans, error, 20, 9, 150, 2)

PysmtTypeError: Invalid type in constant. The type was:<class 'str'>

### Função invert
    - Função invert que recebe a função python que codifica a relação de transição e devolve a relação de transição inversa.

In [6]:
def invert(trans, n_bits):
    return (lambda c, p: trans(p,c, n_bits))

### O algoritmo de "model-checking"

O algoritmo de “model-checking” manipula as fórmulas $\;\mathsf{R}_n\;\equiv\; \mathsf{I}\,\land\,\mathsf{T}^n\;$ e $\;\mathsf{U}_m\equiv\; \mathsf{E}\,\land\,\mathsf{B}^m\;$ fazendo crescer os índices $\;n,m\;$. Neste exemplo, os índices $\;n,m\;$ crescem de acordo com o `input` do utilizador.

Para auxiliar na implementação deste algoritmo, começamos por definir duas funções.
A função `rename` renomeia uma fórmula (sobre um estado) de acordo com um dado estado. 
A função `same` testa se dois estados são iguais.

In [7]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([state1[x] == state2[x] for x in state1])

### O algoritmo de "model-checking"


In [8]:
# Falta ver o porquê do pysmt bater mal

def model_checking(vars,init,trans,error,N, M, n_bits, a, b):
    s = Solver()

    # Criar todos os estados que poderão vir a ser necessários.
    X = [genState(vars, 'X', i, n_bits) for i in range(N+1)]
    Y = [genState(vars, 'Y', i, n_bits) for i in range(M+1)]
        
    
    (n,m) = (1,1)
    command = 0
    while command != 3:
        Tn = And([trans(X[i], X[i+1], n_bits) for i in range(n)])
        I = init(X[0], a, b) 
        Rn = And(I, Tn)
            
        Bm = And([invert(trans, n_bits)(Y[i], Y[i+1]) for i in range(m)])
        E = error(Y[0])
        Um = And(E, Bm)
            
        Vnm = And(Rn, same(X[n], Y[m]), Um)
        s.add([Vnm])
        check = s.check()
        if check == sat:
            print("Unsafe")
            return
        else:                        # Vnm é instatisfazível
            C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
            #C =1
            if C is None:
                print("Interpolant None")
                break
            C0 = rename(C, X[0])
            C1 = rename(C, X[1])
            T = trans(X[0], X[1], n_bits)
            if not s.solve([C0, T, Not(C1)]):   # C é invariante de T
                print("Safe")
                return
            else:
                ### tenta gerar o majorante S
                S = rename(C, X[n])
                while True:
                    A = And(S, trans(X[n], Y[m]))
                    if s.solve([A,Um]):
                        print("Não é possível encontrar um majorante")
                        break
                    else:
                        Cnew = binary_interpolant(A, Um)
                        Cn = rename(Cnew, X[n])
                        if s.solve([Cn, Not(S)]):   # Se Cn -> S não é tautologia
                            S = Or(S, Cn)
                        else:             # S foi encontrado
                            print("Safe")
                            return
        command = int(input("1- Aumentar n\n2- Aumentar m\n3- Sair"))    
                            
        print("unknown")                   


                        

model_checking(['pc', 'x', 'y', 'z'], init, trans, error, 50, 50, 9, 150, 2)            

PysmtTypeError: Invalid type in constant. The type was:<class 'str'>

In [57]:
(n,m) = (1,1)
n = n+1
print(n)

2
